<br>

# Natural Language Generation in `Python`

<br>

## Introduction to Sequential Data

In [21]:
# environment
import requests
import pandas as pd
import numpy as np 
import tensorflow as tf
import tensorflow.compat.v1 as tfc
from keras.layers import SimpleRNN, Dense, Activation, TimeDistributed, LSTM
from keras.models import Sequential
from keras import backend

<br>

### Handling Sequential Data

**sequential data** - any kind of data where the order matters (e.g. text data, time series data, DNA sequences, etc.)  

**word delimiters** - specify the start and end of a name using special start and end tokens  
**start** - `\t`  
**end** - `\n`

In [22]:
url = 'https://assets.datacamp.com/production/repositories/5286/datasets/45e193467da41ae7631b0d4d626c63d832a34cab/names.txt'
names = requests.get( url )
names = names.text.split()
names = [ name.lower() for name in names ]
print( len( names ) )
print( names[0:10] ) 

258000
['john', 'william', 'james', 'charles', 'george', 'frank', 'joseph', 'thomas', 'henry', 'robert']


In [23]:
names_df = pd.DataFrame( names )
names_df.columns = ['name']
names_df.head( 5 )

,name
0,john
1,william
2,james
3,charles
4,george


In [24]:
# start token in front of the name
names_df[ 'name' ] = names_df[ 'name' ].apply( lambda x: '\t' + x )
names_df.head( 5 )

,name
0,\tjohn
1,\twilliam
2,\tjames
3,\tcharles
4,\tgeorge


In [25]:
# end token at the end of the name
names_df[ 'target' ] = names_df[ 'name' ].apply( lambda x: x[1:len(x)] + '\n' )
names_df.head( 5 )

,name,target
0,\tjohn,john\n
1,\twilliam,william\n
2,\tjames,james\n
3,\tcharles,charles\n
4,\tgeorge,george\n


<br>

**Vocabulary** - set of all unique characters used in the dataset

In [26]:
def get_vocabulary( names ):
    """
    Define vocabulary as a set and include start and end tokens
    """
    vocabulary = set( [ '\t', '\n' ] )
    #iterate over all names and all characters of each na,e
    for name in names:
        for c in name:
            if c not in vocabulary:
                vocabulary.add( c )
    return vocabulary

In [27]:
# Sort the vocabulary and assign numbers in order

chars = get_vocabulary( names )
print( chars )
ctoi = { char: idx for idx, char in enumerate( sorted( chars ) ) }
print( ctoi )

{'r', 'p', '\n', 'm', 'u', 'h', 'k', 'e', 'b', 'z', 'g', 'f', '\t', 'x', 'o', 'y', 'n', 'a', 'j', 'i', 't', 'c', 's', 'q', 'v', 'w', 'd', 'l'}
{'\t': 0, '\n': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27}


In [28]:
# sort the inverse: and integer to character mapping

itoc = { idx : char for idx, char in enumerate( sorted( chars ) ) }
print( itoc )

{0: '\t', 1: '\n', 2: 'a', 3: 'b', 4: 'c', 5: 'd', 6: 'e', 7: 'f', 8: 'g', 9: 'h', 10: 'i', 11: 'j', 12: 'k', 13: 'l', 14: 'm', 15: 'n', 16: 'o', 17: 'p', 18: 'q', 19: 'r', 20: 's', 21: 't', 22: 'u', 23: 'v', 24: 'w', 25: 'x', 26: 'y', 27: 'z'}


<br>

### Introduction to Recurrent Neural Network

**feedforward neural networks** - accept a fixed size input and return a fixed size output using a fixed nnumber of hidden layers in between.  
**recurrent neural networks** - in feedforward NN architecture, the inputs are independent; this is not suitable for sequential data where inputs are reliant on context. Recurrant NNs: the history and the current input are used together to create the output.  

RNN for a baby name generator: generate next character given the current. keep track of history so far. continue until the end of the sequence  

**Encoding the characters** - a one-hot encoding the length of the number of characters (vocabulary size)  
**Number of Time steps** - the length of the longest name. predict each sequence as a name of length `max_len`  
**input vector** - initialize as a 3D vector with dims ( num_names, max_lenth +1, length_vocabulary )

In [29]:
# get the length of the longest name

def get_max_len( names ):
    length_list = [ len( name ) for name in names_df[ 'name' ] ]
    max_len = np.max( length_list )
    return max_len

max_len = get_max_len( names_df )
print( max_len )

12


In [30]:
# create a 3D input vector
input_data = np.zeros( (len( names_df.name ), max_len+1, len( chars ) ), dtype='float32' )
print( input_data.shape )
print( input_data[ 0, :, :] )

(258000, 13, 28)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0.

In [31]:
# use the character to integer mappings to fill the input vector

for n_idx, name in enumerate( names_df.name ):
    for c_idx, char in enumerate( name ):
        input_data[ n_idx, c_idx, ctoi[ char ] ] = 1
        
print( input_data[ 0, :, :] )

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0

In [32]:
# initialize and define the target vector

target_data = np.zeros( (len( names_df.name ), max_len+1, len( chars ) ), dtype='float32' )

for n_idx, name in enumerate( names_df.target ):
    for c_idx, char in enumerate( name ):
        target_data[ n_idx, c_idx, ctoi[ char ] ] = 1
        
print( target_data[ 0, :, :] )

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0

In [33]:
# Build and compile an RNN in Keras
model = Sequential()
model.add( SimpleRNN( 50, input_shape=( max_len + 1, len( chars ) ),
                    return_sequences = True ) )
model.add( TimeDistributed( Dense( len( chars), activation = 'softmax' ) ) )
model.compile( loss = 'categorical_crossentropy', optimizer = 'adam' )
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 13, 50)            3950      
_________________________________________________________________
time_distributed (TimeDistri (None, 13, 28)            1428      
Total params: 5,378
Trainable params: 5,378
Non-trainable params: 0
_________________________________________________________________


<br>

### Inference Using Recurrent Neural Network

**Understanding Training**  

* NN: a black box
* Input target pairs (x,y): ideal output y for input x
* the model takes input x $\rightarrow$ some internal processes $\rightarrow$ an output z
* GOAL: reduce the differences between actual output z and the ideal output y
* Training - adjust the internal model parameters to achieve the goal
* After training the actual output should be more similar to the ideal output

<br>

In [34]:
# Traing the RNN

model.fit( input_data, target_data, batch_size = 128, epochs = 15 )

Epoch 1/15
2016/2016 [==============================] - 12s 3ms/step - loss: 1.2404
Epoch 2/15
2016/2016 [==============================] - 6s 3ms/step - loss: 1.0316
Epoch 3/15
2016/2016 [==============================] - 6s 3ms/step - loss: 0.9900
Epoch 4/15
2016/2016 [==============================] - 6s 3ms/step - loss: 0.9648
Epoch 5/15
2016/2016 [==============================] - 6s 3ms/step - loss: 0.9478
Epoch 6/15
2016/2016 [==============================] - 6s 3ms/step - loss: 0.9345
Epoch 7/15
2016/2016 [==============================] - 6s 3ms/step - loss: 0.9226
Epoch 8/15
2016/2016 [==============================] - 6s 3ms/step - loss: 0.9146
Epoch 9/15
2016/2016 [==============================] - 6s 3ms/step - loss: 0.9087
Epoch 10/15
2016/2016 [==============================] - 6s 3ms/step - loss: 0.9018
Epoch 11/15
2016/2016 [==============================] - 7s 3ms/step - loss: 0.8986
Epoch 12/15
2016/2016 [==============================] - 7s 3ms/step - loss: 0.8947


<br>

where:  

* **batch size** - number of sample after which the paramters are adjusted
* **epoch** - number of times to iterate over the full dataset

<br>

In [38]:
# Predict the first character

# initialize the first character of the sequence
output_seq = np.zeros( ( 1, max_len+1, len( chars ) ) )
output_seq[ 0, 0, ctoi['\t'] ] = 1

# probability distribution for the next character
probs = model.predict_proba( output_seq, verbose = 0 )[ :,1,: ]
print( probs )

[[1.32839573e-09 1.01555976e-04 1.98415011e-01 5.17036766e-04
  1.87392649e-03 2.21476104e-04 1.45850286e-01 3.36497105e-05
  1.88017308e-04 5.56050614e-03 2.24777922e-01 6.55317708e-05
  2.03310192e-04 1.04195829e-02 8.83700792e-04 2.87403614e-04
  1.65073812e-01 2.42132621e-04 8.02669310e-05 5.23641296e-02
  8.23235547e-04 2.45434046e-03 1.61420986e-01 3.11078969e-03
  2.56880256e-03 2.16547269e-05 2.18079090e-02 6.33075717e-04]]


In [39]:
# sample the vocabulary to randomly generate a first character
first_char = np.random.choice( sorted( list( chars ) ), replace = False, p = probs.reshape( 28 ) )

# insert the first character into a sequence
output_seq[ 0, 1, ctoi[ first_char ] ] = 1

# sample from probability distribution
probs = model.predict_proba( output_seq, verbose=0 )[:,1,:]
second_char = np.random.choice( sorted( list( chars ) ), replace=False, p = probs.reshape( 28 ) )
print( 'first char: ', first_char, '\nsecond char: ', second_char )

first char:  a 
second char:  l


In [40]:
# a function to generate names

def generate_names( n ):
    for i in range( 0, n ):
        stop = False
        counter = 1
        name = ''
        # initialize the fisrt char of the output sequence
        output_seq = np.zeros( ( 1, max_len+1, 28 ) )
        output_seq[ 0, 0, ctoi[ '\t' ] ] = 1
        # continue until a newline is generated or max number of characters is reached
        while stop == False and counter < 10:
            # get the prob distribution for the next character
            probs = model.predict_proba( output_seq, verbose=0 )[ :,counter-1,: ]
            # sample vocabulary to get the most probable next charachter
            c = np.random.choice( sorted( list( chars ) ), replace = False, p=probs.reshape( 28 ) )
            if c == '\n':
                stop = True
            else:
                name = name + c
                output_seq[ 0, counter, ctoi[c] ] = 1
                counter += 1
    return name

In [42]:
lens = np.random.randint( low = 5, high = 12, size = 10 )

for alen in lens:
    a_name = generate_names( alen )
    print( a_name )

randitt
judta
sod
trent
annkalvi
harronne
judre
reokieva
shawes
allyno


<br>

## Write Like Shakespeare

<br>

### Limitations of Recurrent Neural Networks

RNNs are not the best for handling long sequences. We will need another approach.  

**Simple neural networks** can be thought of as nodes arranged into layers where nodes in different layers are connected by weights. A node/neuron takes in the weights from the previous layer and performs a linear transformation to combine them. Then, a nonlinear 'activation' transformation is applied to create the ouput. In theory, the combination of linear followed by nonlinear transformations makes the network very powerful and it could be able to approximate just about any functions.  

**Gradients and Training**  
Error: squared difference of the actual output and the predicted output
$$E = \sum e_i = \sum(y_i-\hat{y}_i)^2$$
Gradient: rate of change of error with respect to the weights
$$g_i = \frac{\Delta E}{\Delta w_i} = \frac{\partial E}{\partial w_i}$$
training is nothing but adjusting the weights by the gradient fraction to reducing the error
$$w_i = w_i-\eta * \frac{\partial E}{\partial w_i}$$
Learning Rate ($\eta$): factor by which to adjust the weights  

Gradients in the output layer can be found by differentiation and other layers by an application of the Chain Rule. Gradients are the product of many gradient values from subsequent time-steps. The gradient that is calculated at the output layer is backpropagated to previous layers where the gradients typically become smaller and smaller than at earlier timepoints in training.  
**Vanishing Gradients** - if gradients are very close or equal to zero, then the model will stop learning  
**Exploding Gradients** - of gradients become too large and increase, the value will continue to increase with backpropagation.  
**Solutions**: use a fixed number of time-steps to avoid vanishing gradients and/or clip gradients to avoid explosion. However, these will result in suboptimal traing and reduce performance.

<br>

In [ ]:
# Create a sequential model
simple_model = Sequential()

# Create a dense layer of 12 units
simple_model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))

# Create a dense layer of 8 units
simple_model.add(Dense(8, kernel_initializer='uniform', activation='relu'))

# Create a dense layer of 1 unit
simple_model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

# Compile the model and get gradients
simple_model.compile(loss='binary_crossentropy', optimizer='adam')

simple_model.summary()

inputs = tf.ones((8,8))

with tf.GradientTape() as tape:
    preds = simple_model( inputs )
gradients = tape.gradient(preds, simple_model.trainable_weights)
print( gradients )

<br>

### Introduction to Long Short Term Memeory (LSTM)

**Long-term dependencies**  

* short-term: The birds are flying in the ___
* long-term: I was born in Germany. (many sentences) I can speak ___

RNNs are good for short-term memory, but struggle with long term due to vanishing and exploding gradients  
LSTM uses an additionaly state to capture longer-term memory  

In [8]:
url = 'https://assets.datacamp.com/production/repositories/5286/datasets/2b130693c9bd45c528b60fa9efbf5148a3ff14e5/shakespear.txt'

text = requests.get( url )
text = text.text.lower()
print( len( text ) )
print( text[0:10] ) 

99993
that, poor


In [9]:
vocabulary = sorted( set( text ) )
vocabulary = ['\n',' ','!',"'",',','-','.',':',';','?','a','b','c','d','e','f','g','h','i','j','k','l','m','n',
              'o','p','q','r','s','t','u','v','w','x','y','z']
char_to_idx = dict( (char,idx) for idx, char in enumerate( vocabulary ) )
idx_to_char = dict( (idx,char) for idx, char in enumerate( vocabulary ) )

In [10]:
input_data = []
target_data = []
maxlen = 40
for i in range( 0, len( text ) - maxlen ):
    input_data.append( text[i:i+maxlen])
    target_data.append(text[i+maxlen])
    
# Print number of sequences in input data
print('No of Sequences:', len(input_data))

No of Sequences: 99953


In [11]:
print( input_data[0:3])
print( target_data[0:3])

["that, poor contempt, or claim'd thou sle", "hat, poor contempt, or claim'd thou slep", "at, poor contempt, or claim'd thou slept"]
['p', 't', ' ']


In [12]:
#create input and target vectors
x = np.zeros((len(input_data), maxlen, len( vocabulary)), dtype='float32')
y = np.zeros((len(target_data), len( vocabulary)), dtype='float32')

In [13]:
#iterate over the sequences
for s_idx, sequence in enumerate( input_data ):
    for idx, char in enumerate( sequence ):
        x[ s_idx, idx, char_to_idx[ char ] ] = 1
    y[ s_idx, char_to_idx[ target_data[s_idx] ] ] = 1

In [14]:
# create the LSTM network in Keras
lstmmod = Sequential()
lstmmod.add( LSTM( 128, input_shape=(maxlen, len(vocabulary)) ) )
lstmmod.add( Dense( len(vocabulary), activation='softmax' ) )
lstmmod.compile( loss='categorical_crossentropy', optimizer='adam' )
lstmmod.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               84480     
_________________________________________________________________
dense_1 (Dense)              (None, 36)                4644      
Total params: 89,124
Trainable params: 89,124
Non-trainable params: 0
_________________________________________________________________


<br>

### Inference Using LSTM

How LSTM can be trained and used for prediction  
use a validation split to keep samples aside

<br>

In [15]:
# fit the LSTM
lstmmod.fit( x, y, batch_size = 64, epochs = 20, validation_split = 0.2 )

Epoch 1/20
1250/1250 [==============================] - 38s 21ms/step - loss: 2.8422 - val_loss: 2.2906
Epoch 2/20
1250/1250 [==============================] - 24s 19ms/step - loss: 2.2472 - val_loss: 2.1373
Epoch 3/20
1250/1250 [==============================] - 24s 19ms/step - loss: 2.0921 - val_loss: 2.0301
Epoch 4/20
1250/1250 [==============================] - 24s 19ms/step - loss: 1.9938 - val_loss: 1.9647
Epoch 5/20
1250/1250 [==============================] - 24s 19ms/step - loss: 1.9103 - val_loss: 1.9078
Epoch 6/20
1250/1250 [==============================] - 30s 24ms/step - loss: 1.8439 - val_loss: 1.8682
Epoch 7/20
1250/1250 [==============================] - 26s 21ms/step - loss: 1.7939 - val_loss: 1.8352
Epoch 8/20
1250/1250 [==============================] - 24s 19ms/step - loss: 1.7407 - val_loss: 1.8046
Epoch 9/20
1250/1250 [==============================] - 24s 19ms/step - loss: 1.6900 - val_loss: 1.7778
Epoch 10/20
1250/1250 [==============================] - 24s 20m

In [16]:
sentence = "that, poor contempt, or claim'd thou sle"

#one hot encode the sentence
X_test = np.zeros( ( 1, maxlen, len( vocabulary ) ) )
for t, char in enumerate( sentence ):
    X_test[ 0, t, char_to_idx[ char ] ] = 1

In [17]:
# predeict the next character

preds = lstmmod.predict( X_test, verbose=0 )
prob_next_char = preds[0]
next_index = np.argmax( prob_next_char )
next_char = idx_to_char[ next_index ]
print( next_char )

a


In [47]:
def generate_text( sentence, n ):
    generated = sentence
    for i in range( n ):
        x_pred = np.zeros((1,maxlen,len(vocabulary)))
        for t,char in enumerate( sentence ):
            x_pred[0,t,char_to_idx[char]]=1.
        preds = lstmmod.predict( x_pred,verbose=0)[0]
        next_index = np.argmax( preds )
        next_char = idx_to_char[ next_index ]
        sentence = sentence[1:]+next_char
        generated += next_char
    return generated

In [20]:
res = generate_text( sentence, 400 )
print( res )

that, poor contempt, or claim'd thou slead tone and the starn
as he was stay'd and the starn
and so stor out of no soul be with the partions,
that he doust of discourse and in the partions the starn
as he was stay'd and the starn
and so stor out of no soul be with the partions,
that he doust of discourse and in the partions the starn
as he was stay'd and the starn
and so stor out of no soul be with the partions,
that he doust of discour


In [49]:
sent2 = 'upon our wrongs despised, i will not'
res = generate_text( sent2, 250 )
print( res )

upon our wrongs despised, i will note
 'n tasye  ' oirrr eoias noeeeesaa:,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,oaoaaauu eou eii eoee eee eae
iatoaa iore
ne ieee eai  ireeaaa
ne aoeei caei e neioeisoa
le ieee eaee eiiau iyeaa
ree iat
 toe le eeiat ieeeereisoaa,oa ne oee eoe
ieeee eii eoee eoe
